In [ ]:
pip install boto3 opencv-python-headless pandas


In [ ]:
import os
import cv2
import boto3
import pandas as pd
import numpy as np

# Lista para almacenar los resultados
results = []

# Configura las credenciales de AWS
aws_access_key_id = 'AWS_ACCESS_KEY_ID'
aws_secret_access_key = 'AWS_SECRET_ACCESS_KEY'
aws_region = 'us-west-2'  # Cambia esto a tu región de S3
bucket_name = 'BUCKET_NAME'

# Carpeta en S3 que deseas analizar
s3_folder = 'users/130510'

# Inicializa el cliente de S3
s3_client = boto3.client(
    's3',
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=aws_region
)

# Función para determinar el porcentaje de pantalla negra y la duración de un video
def detect_black_screen_and_duration(video_path):
    video = cv2.VideoCapture(video_path)
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    black_frames = 0
    frame_rate = int(video.get(cv2.CAP_PROP_FPS))
    duration = total_frames / frame_rate
    duration = round(duration, 2)

    while True:
        ret, frame = video.read()
        if not ret:
            break

        # Convierte el fotograma a escala de grises
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Define un umbral para considerar que un fotograma es negro
        # Puedes ajustar este umbral según tus necesidades
        threshold = 30

        # Aplica un umbral binario al fotograma
        _, binary_frame = cv2.threshold(gray_frame, threshold, 255, cv2.THRESH_BINARY)

        # Cuenta los píxeles negros en el fotograma binario
        black_pixels = np.count_nonzero(binary_frame == 0)

        # Si más del 90% de los píxeles son negros, consideramos el fotograma como negro
        if (black_pixels / binary_frame.size) > 0.9:
            black_frames += 1

    video.release()

    # Calcula el porcentaje de fotogramas negros
    black_screen_percentage = (black_frames / total_frames) * 100

    return black_screen_percentage, duration

# Recorre los objetos en la carpeta de S3 y almacena los resultados
def analyze_s3_folder(bucket_name, s3_folder):
    s3_objects = s3_client.list_objects(Bucket=bucket_name, Prefix=s3_folder)
    for s3_object in s3_objects.get('Contents', []):
        s3_key = s3_object['Key']
        if s3_key.endswith('.mp4'):
            print(f"Analizando: {s3_key}")
            temp_video_path = '/tmp/temp_video.mp4'  # Ruta temporal para descargar el video
            s3_client.download_file(bucket_name, s3_key, temp_video_path)
            black_screen_percentage, duration = detect_black_screen_and_duration(temp_video_path)
            os.remove(temp_video_path)
            print(f"Black Screen detected: {black_screen_percentage}% Duracion: {duration}")
            print("---")
            results.append({'Archivo': s3_key, 'Black Screen detected: ': f"{black_screen_percentage}%" , 'Duracion' : duration})

# Llama a la función para analizar la carpeta de S3
analyze_s3_folder(bucket_name, s3_folder)

# Crea un DataFrame de pandas con los resultados
df = pd.DataFrame(results)

# Muestra el DataFrame
print("Tabla de Datos:")
print(df)

# Guarda el DataFrame en un archivo CSV
df.to_csv('resultados.csv', index=False)
